In [1]:
!apt-get install python-opengl -y 
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install -U gym
!pip install -U ale-py


'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from ale_py import ALEInterface
from ale_py.roms import Pong

ale = ALEInterface()
ale.loadROM(Pong)

In [3]:
%matplotlib inline
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 144)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    plt.close(anim._fig)
    display(HTML(anim.to_jshtml()))

In [4]:
import gym
env = gym.make('ALE/Pong-v5')

C:\Users\Aravind\anaconda3\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\Aravind\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
env.action_space

Discrete(6)

In [7]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [8]:
# Run a demo of the environment
observation = env.reset()
cumulated_reward = 0

frames = []
for t in range(1000):
#     print(observation)
    frames.append(env.render(mode = 'rgb_array'))
    # very stupid agent, just makes a random action within the allowd action space
    action = env.action_space.sample()
#     print("Action: {}".format(t+1))    
    observation, reward, done, info = env.step(action)
#     print(reward)
    cumulated_reward += reward
    if done:
        print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
        break
print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))

env.close()

C:\Users\Aravind\anaconda3\lib\site-packages\gym\core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
C:\Users\Aravind\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
C:\Users\Aravind\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Episode finished without success, accumulated reward = -20.0


In [9]:
display_frames_as_gif(frames)

In [10]:
def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):
  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('resetting env. episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

In [11]:
import numpy as np

# model initialization
H = 400 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)

# import pickle
# model = pickle.load(open('model.pkl', 'rb'))

In [12]:
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
# learning_rate = 1e-4
learning_rate = 1e-3
 
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

In [ ]:
%time hist1 = train_model(env, model, total_episodes=4000)

C:\Users\Aravind\AppData\Local\Temp/ipykernel_8272/1054945630.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return I.astype(np.float).ravel()


resetting env. episode 1.000000, reward total was -20.000000. running mean: -20.000000
resetting env. episode 2.000000, reward total was -20.000000. running mean: -20.000000
resetting env. episode 3.000000, reward total was -20.000000. running mean: -20.000000
resetting env. episode 4.000000, reward total was -20.000000. running mean: -20.000000
resetting env. episode 5.000000, reward total was -21.000000. running mean: -20.010000
resetting env. episode 6.000000, reward total was -21.000000. running mean: -20.019900
resetting env. episode 7.000000, reward total was -21.000000. running mean: -20.029701
resetting env. episode 8.000000, reward total was -21.000000. running mean: -20.039404
resetting env. episode 9.000000, reward total was -21.000000. running mean: -20.049010
resetting env. episode 10.000000, reward total was -21.000000. running mean: -20.058520
resetting env. episode 11.000000, reward total was -21.000000. running mean: -20.067935
resetting env. episode 12.000000, reward 

resetting env. episode 95.000000, reward total was -19.000000. running mean: -19.998808
resetting env. episode 96.000000, reward total was -20.000000. running mean: -19.998820
resetting env. episode 97.000000, reward total was -21.000000. running mean: -20.008832
resetting env. episode 98.000000, reward total was -18.000000. running mean: -19.988743
resetting env. episode 99.000000, reward total was -18.000000. running mean: -19.968856
resetting env. episode 100.000000, reward total was -20.000000. running mean: -19.969167
resetting env. episode 101.000000, reward total was -18.000000. running mean: -19.949476
resetting env. episode 102.000000, reward total was -21.000000. running mean: -19.959981
resetting env. episode 103.000000, reward total was -19.000000. running mean: -19.950381
resetting env. episode 104.000000, reward total was -20.000000. running mean: -19.950877
resetting env. episode 105.000000, reward total was -21.000000. running mean: -19.961368
resetting env. episode 106

resetting env. episode 188.000000, reward total was -19.000000. running mean: -19.437183
resetting env. episode 189.000000, reward total was -20.000000. running mean: -19.442811
resetting env. episode 190.000000, reward total was -20.000000. running mean: -19.448383
resetting env. episode 191.000000, reward total was -19.000000. running mean: -19.443899
resetting env. episode 192.000000, reward total was -19.000000. running mean: -19.439460
resetting env. episode 193.000000, reward total was -20.000000. running mean: -19.445066
resetting env. episode 194.000000, reward total was -20.000000. running mean: -19.450615
resetting env. episode 195.000000, reward total was -18.000000. running mean: -19.436109
resetting env. episode 196.000000, reward total was -18.000000. running mean: -19.421748
resetting env. episode 197.000000, reward total was -19.000000. running mean: -19.417531
resetting env. episode 198.000000, reward total was -20.000000. running mean: -19.423355
resetting env. episod

resetting env. episode 281.000000, reward total was -18.000000. running mean: -19.169671
resetting env. episode 282.000000, reward total was -21.000000. running mean: -19.187975
resetting env. episode 283.000000, reward total was -18.000000. running mean: -19.176095
resetting env. episode 284.000000, reward total was -18.000000. running mean: -19.164334
resetting env. episode 285.000000, reward total was -21.000000. running mean: -19.182691
resetting env. episode 286.000000, reward total was -16.000000. running mean: -19.150864
resetting env. episode 287.000000, reward total was -18.000000. running mean: -19.139355
resetting env. episode 288.000000, reward total was -18.000000. running mean: -19.127962
resetting env. episode 289.000000, reward total was -21.000000. running mean: -19.146682
resetting env. episode 290.000000, reward total was -20.000000. running mean: -19.155215
resetting env. episode 291.000000, reward total was -18.000000. running mean: -19.143663
resetting env. episod

...

resetting env. episode 653.000000, reward total was -15.000000. running mean: -16.974156
resetting env. episode 654.000000, reward total was -19.000000. running mean: -16.994415
resetting env. episode 655.000000, reward total was -20.000000. running mean: -17.024471
resetting env. episode 656.000000, reward total was -20.000000. running mean: -17.054226
resetting env. episode 657.000000, reward total was -18.000000. running mean: -17.063684
resetting env. episode 658.000000, reward total was -16.000000. running mean: -17.053047
resetting env. episode 659.000000, reward total was -20.000000. running mean: -17.082516
resetting env. episode 660.000000, reward total was -15.000000. running mean: -17.061691
resetting env. episode 661.000000, reward total was -16.000000. running mean: -17.051074
resetting env. episode 662.000000, reward total was -19.000000. running mean: -17.070563
resetting env. episode 663.000000, reward total was -16.000000. running mean: -17.059858
resetting env. episod

resetting env. episode 746.000000, reward total was -13.000000. running mean: -16.849514
resetting env. episode 747.000000, reward total was -17.000000. running mean: -16.851018
resetting env. episode 748.000000, reward total was -18.000000. running mean: -16.862508
resetting env. episode 749.000000, reward total was -8.000000. running mean: -16.773883
resetting env. episode 750.000000, reward total was -19.000000. running mean: -16.796144
resetting env. episode 751.000000, reward total was -14.000000. running mean: -16.768183
resetting env. episode 752.000000, reward total was -17.000000. running mean: -16.770501
resetting env. episode 753.000000, reward total was -18.000000. running mean: -16.782796
resetting env. episode 754.000000, reward total was -20.000000. running mean: -16.814968
resetting env. episode 755.000000, reward total was -19.000000. running mean: -16.836818
resetting env. episode 756.000000, reward total was -16.000000. running mean: -16.828450
resetting env. episode

resetting env. episode 839.000000, reward total was -13.000000. running mean: -15.403087
resetting env. episode 840.000000, reward total was -17.000000. running mean: -15.419056
resetting env. episode 841.000000, reward total was -9.000000. running mean: -15.354865
resetting env. episode 842.000000, reward total was -15.000000. running mean: -15.351317
resetting env. episode 843.000000, reward total was -13.000000. running mean: -15.327804
resetting env. episode 844.000000, reward total was -15.000000. running mean: -15.324526
resetting env. episode 845.000000, reward total was -14.000000. running mean: -15.311280
resetting env. episode 846.000000, reward total was -14.000000. running mean: -15.298167
resetting env. episode 847.000000, reward total was -17.000000. running mean: -15.315186
resetting env. episode 848.000000, reward total was -15.000000. running mean: -15.312034
resetting env. episode 849.000000, reward total was -13.000000. running mean: -15.288914
resetting env. episode

resetting env. episode 932.000000, reward total was -10.000000. running mean: -14.169311
resetting env. episode 933.000000, reward total was -6.000000. running mean: -14.087618
resetting env. episode 934.000000, reward total was -18.000000. running mean: -14.126742
resetting env. episode 935.000000, reward total was -19.000000. running mean: -14.175475
resetting env. episode 936.000000, reward total was -18.000000. running mean: -14.213720
resetting env. episode 937.000000, reward total was -12.000000. running mean: -14.191583
resetting env. episode 938.000000, reward total was -14.000000. running mean: -14.189667
resetting env. episode 939.000000, reward total was -10.000000. running mean: -14.147770
resetting env. episode 940.000000, reward total was -12.000000. running mean: -14.126292
resetting env. episode 941.000000, reward total was -13.000000. running mean: -14.115029
resetting env. episode 942.000000, reward total was -16.000000. running mean: -14.133879
resetting env. episode

resetting env. episode 1025.000000, reward total was -13.000000. running mean: -13.118269
resetting env. episode 1026.000000, reward total was -11.000000. running mean: -13.097087
resetting env. episode 1027.000000, reward total was -7.000000. running mean: -13.036116
resetting env. episode 1028.000000, reward total was -17.000000. running mean: -13.075755
resetting env. episode 1029.000000, reward total was -14.000000. running mean: -13.084997
resetting env. episode 1030.000000, reward total was -10.000000. running mean: -13.054147
resetting env. episode 1031.000000, reward total was -10.000000. running mean: -13.023606
resetting env. episode 1032.000000, reward total was -14.000000. running mean: -13.033370
resetting env. episode 1033.000000, reward total was -9.000000. running mean: -12.993036
resetting env. episode 1034.000000, reward total was -15.000000. running mean: -13.013106
resetting env. episode 1035.000000, reward total was -16.000000. running mean: -13.042975
resetting en

In [ ]:
play_game(env, model)